In [1]:
import os, re
import numpy as np
import pandas as pd

In [2]:
os.chdir('..')

---

In [99]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'compustat.tsv'),
                 sep = '\t')

add historic header variables

In [100]:
comphist = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'comphist_prepared.tsv'),
                       sep = '\t')

In [101]:
df = df.merge(right = comphist, on = ['gvkey', 'fyear'], how = 'left')

add header variables and replace missing values on the historic variables by the header; then drop header variables

In [102]:
comp_header = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'compustat_header.tsv'),
                          sep = '\t')
comp_header = comp_header.drop(columns = 'loc')

In [103]:
df = df.merge(right = comp_header, on = 'gvkey', how = 'left')

In [104]:
for var in ['fic', 'sic', 'cik']:
    df[f'h{var}'] = df[f'h{var}'].fillna(df[var])

In [105]:
df = df.drop(columns = ['fic', 'sic', 'cik'])

add CRSP PERMNO (based on CCM)

In [106]:
ccm = pd.read_csv(os.path.join(os.getcwd(), '1_data', 'compustat_crsp_link.tsv'),
                  sep = '\t')

In [107]:
df = df.merge(right = ccm, on = ['gvkey', 'fyear'], how = 'left')

In [108]:
df = df.rename(columns = {'lpermno' : 'permno'})

add IBES ticker (based on CRSP-IBES link)

In [109]:
ibes = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'crsp_ibes_link_final.tsv'),
                   sep = '\t')

In [110]:
ibes = ibes.rename(columns = {'ticker' : 'ibes_ticker'})

In [111]:
df = df.merge(right = ibes, on = ['permno', 'fyear'], how = 'left')

load VA data and merge to the Compustat records; using the cik-fyear

In [112]:
va_info = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_info.tsv'),
                      sep = '\t')

In [113]:
va_info = va_info.drop_duplicates(subset = ['cik', 'fyear'])

In [114]:
va_info = va_info.rename(columns = {'cik' : 'hcik', 'sic' : 'sic_from_sec'})

In [115]:
df = df.merge(right = va_info, on = ['hcik', 'fyear'], how = 'left')

add valuation allowance data

In [116]:
va = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'va_data_clean.tsv'),
                 sep = '\t')

In [117]:
df = df.merge(right = va[['adsh', 'tag', 'va', 'zero_inferred']], on = 'adsh', how = 'left')

add DTA (gross)

In [118]:
dta = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'dta_gross.tsv'),
                  sep = '\t')

In [120]:
df = df.merge(right = dta, on = 'adsh', how = 'left')

save to disk

In [122]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'sample_raw.tsv'),
          sep = '\t', index = False)

---

### OLD

load valuation allowance data

In [261]:
df = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'sample_meta_intermediate.tsv'),
                 sep = '\t')

load historic Compustat headers, and replace with most recent header if historic is missing

In [262]:
comphist = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'comphist_prepared.tsv'),
                       sep = '\t')

In [263]:
comp_header = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'compustat_header.tsv'),
                          sep = '\t')
comp_header = comp_header.drop(columns = 'loc')

In [264]:
comphist = comphist.merge(right = comp_header, how = 'left', on = 'gvkey')

In [265]:
comphist['hfic'] = comphist['hfic'].fillna(comphist['fic'])
comphist['hsic'] = comphist['hsic'].fillna(comphist['sic'])
comphist['hcik'] = comphist['hcik'].fillna(comphist['cik'])

In [266]:
comphist = comphist.drop(columns = ['sic', 'fic', 'cik'])

In [267]:
comphist = comphist.rename(columns = {'hfic' : 'fic',
                                      'hsic' : 'sic',
                                      'hcik' : 'cik'})

In [268]:
df = df.rename(columns = {'sic' : 'sic_from_sec'})

In [269]:
df = df.merge(right = comphist, on = ['cik', 'fyear'], how = 'left')

In [270]:
# if company is missing on comphist, it would not be included
# therefore: for the observations where an obsveration is missing in the header file
# and therefore gvkey is N/A in df up until here,
# use fic, sic and gvkey information from compustat header

df = df.merge(right = comp_header.rename(columns = {'fic' : 'fic_header',
                                                    'sic' : 'sic_header',
                                                    'gvkey' : 'gvkey_header'}),
              how = 'left', on = 'cik')

In [271]:
df['fic'] = df['fic'].fillna(df['fic_header'])

In [272]:
df['gvkey'] = df['gvkey'].fillna(df['gvkey_header'])

In [273]:
df['sic'] = df['sic'].fillna(df['sic_header'])

In [274]:
# if historic and header SIC is missing, take the SIC from the SEC
df['sic'] = df['sic'].fillna(df['sic_from_sec'])

add CRSP PERMNO (based on CCM)

In [275]:
ccm = pd.read_csv(os.path.join(os.getcwd(), '1_data', 'compustat_crsp_link.tsv'),
                  sep = '\t')

In [276]:
df = df.merge(right = ccm, on = ['gvkey', 'fyear'], how = 'left')

In [277]:
df = df.rename(columns = {'lpermno' : 'permno'})

add IBES ticker (based on CRSP-IBES link)

In [278]:
ibes = pd.read_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'crsp_ibes_link_final.tsv'),
                   sep = '\t')

In [279]:
ibes = ibes.rename(columns = {'ticker' : 'ibes_ticker'})

In [280]:
df = df.merge(right = ibes, on = ['permno', 'fyear'], how = 'left')

drop if gvkey missing

In [281]:
df = df.loc[df['gvkey'].notna()]

In [282]:
df = df.sort_values(by = ['gvkey', 'fyear'])

drop remaining duplicates

In [283]:
df = df.drop_duplicates(subset = ['gvkey', 'fyear'], keep = False)

reorder columns

In [284]:
df = df[['gvkey', 'fyear'] + [c for c in list(df.columns) if c not in ['gvkey', 'fyear']]]

In [287]:
not_existing = df.loc[~df['gvkey'].isin(test['gvkey'])]

In [129]:
df.to_csv(os.path.join(os.getcwd(), '3_pipeline', '1_intermediate', 'sample_meta_final.tsv'),
          sep = '\t',
          index = False)